#### Merging data, dealing with missing data in pandas

In [1]:
import pandas as pd

In [2]:
#import the data and view
population =pd.read_csv('state-population.csv')
areas= pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

print(population.head())
print(areas.head())
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [3]:
#using the set_index key words
pd.merge(population, abbrevs, how='outer', left_index = True, right_index= True).head(2)

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alaska,AK


* this uses the index to merge the abbreviated 'state/region' column with the 'abbreviation' column. Though it gives us an output, it is a wrong output because the indexes clarify the program on taking both tables but not on arranging the states/region according to their abbrevs. thats why AL is represented as AK below Alaska, instead of Alabama.

* also it is noteworthy that if we use the how='outer' because pd.merge() automatically takes the inner join of tables and between the population and abbrevs table, the abbreviations are common but are only representative meanings of the states and they'd both be taken on the 50 observations in the abbrevs table. instead of the actual 2544 obervations in the popn data

In [4]:
#Now take a look at this.
merged= pd.merge(population, abbrevs, how='outer', left_on='state/region', right_on= 'abbreviation')
#Alabama is aptly represent and the others are too

In [5]:
merged = merged.drop('abbreviation', axis=1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [6]:
#check for duplicates and null values
sum(merged.duplicated())

0

In [7]:
merged.isnull().any()
merged.isnull().sum()

state/region     0
ages             0
year             0
population      20
state           96
dtype: int64

In [8]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [9]:
merged.loc[merged['state'].isnull(),'state/region'].unique()
#check the unique 'state' that is null and its representative 
#in the state/region aspect

array(['PR', 'USA'], dtype=object)

In [10]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [12]:
final = pd.merge(merged, areas, on='state', how= 'left')
#this attaches the data based on the number of observations/features in
#the merged data instead of the ones on the areas column
#the areas column has some missing data for some observations in the merged data
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [13]:
#let's check for any null values 
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [14]:
#to check for how many null values are there...
final.isnull().sum()

state/region      0
ages              0
year              0
population       20
state             0
area (sq. mi)    48
dtype: int64

In [15]:
final[final['population'].isnull()].head()

,state/region,ages,year,population,state,area (sq. mi)
2448,PR,under18,1990,NaN,Puerto Rico,3515.0
2449,PR,total,1990,NaN,Puerto Rico,3515.0
2450,PR,total,1991,NaN,Puerto Rico,3515.0
2451,PR,under18,1991,NaN,Puerto Rico,3515.0
2452,PR,total,1993,NaN,Puerto Rico,3515.0


In [16]:
final[final['area (sq. mi)'].isnull()].head()

,state/region,ages,year,population,state,area (sq. mi)
2496,USA,under18,1990,64218512.0,United States,NaN
2497,USA,total,1990,249622814.0,United States,NaN
2498,USA,total,1991,252980942.0,United States,NaN
2499,USA,under18,1991,65313018.0,United States,NaN
2500,USA,under18,1992,66509177.0,United States,NaN


In [17]:
final['state'][final['area (sq. mi)'].isnull()].unique()\
#to check what state has missing values

array(['United States'], dtype=object)

In [18]:
final.dropna(inplace=True)
#dropping those rows

In [19]:
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


In [20]:
#query() will select the part of the data that we need
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [21]:
#setting the state as the primary key
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [22]:
#sort the data in a descending order
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [23]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64